In [ ]:
# default_exp etl

# etl

> Utilerias para extraer y transformar datos sobre delincuencia en la CDMX.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
import glob
import itertools
from pathlib import Path
import pandas as pd
import geopandas as gpd
from datetime import timedelta, date, datetime
import seaborn as sns
import requests

## get_carpetas_from_api

In [ ]:
# export
def get_carpetas_from_api(limit=100):
    """Baja los primeros `limit` registros en la base de Carpetas de Investigación."""
    url = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=48fcb848-220c-4af0-839b-4fd8ac812c0f&limit=5'
    r = requests.get(url, allow_redirects=True)
    records = r.json()['result']['records'] 
    records = pd.DataFrame(records)
    return records

In [ ]:
carpetas = get_carpetas_from_api()
assert type(carpetas) == pd.core.frame.DataFrame